In [ ]:
#TO DO:

# ordinal 
# categorical+continuous

# feature interaction

# batch normalization

In [1]:
import gzip
import os
import tempfile

import numpy as np
import pandas as pd
# from six.moves import urllib
# from six.moves import xrange  # pylint: disable=redefined-builtin


import tensorflow as tf
from tensorflow.contrib.learn.python.learn.datasets.mnist import read_data_sets
from tensorflow.examples.tutorials.mnist import input_data
from tensorflow.examples.tutorials.mnist import mnist
from tensorflow.contrib import rnn


import math
import random
import time
import datetime

from keras.utils import np_utils

from sklearn import preprocessing



Couldn't import dot_parser, loading of dot files will not be possible.


Using Theano backend.


In [2]:
# load data

files_list=["../dataset/dataset_demo/xtrain.csv", \
            "../dataset/dataset_demo/xtest.csv",\
            "../dataset/dataset_demo/ytrain.csv", \
            "../dataset/dataset_demo/ytest.csv"]

xtrain_df=pd.read_csv( files_list[0] ,sep=',', header=None)
xtest_df=pd.read_csv( files_list[1] ,sep=',', header=None)
ytrain_df=pd.read_csv( files_list[2] ,sep=',', header=None)
ytest_df=pd.read_csv( files_list[3] ,sep=',', header=None)

print xtrain_df.shape, xtest_df.shape, ytrain_df.shape, ytest_df.shape

(6938, 186) (771, 186) (6938, 1) (771, 1)


In [3]:
# continuous and categorical feature split

# features on columns [0:split_idx] are continuous
# the rest are categorical
def conti_cate_split(dta_df, split_idx):
    
    col_cnt= dta_df.shape[1]
    conti_cols= range(split_idx)
    dis_cols=range(split_idx, col_cnt)
    
    return dta_df[conti_cols], dta_df[dis_cols]

def conti_normalization_train_dta(dta_df):
    
    return preprocessing.scale(dta_df)

def conti_normalization_test_dta(dta_df, train_df):
    
    mean_dim = np.mean(train_df, axis=0)
    std_dim = np.std(train_df, axis=0)
        
    df=pd.DataFrame()
    cols = train_df.columns
    idx=0
    
    for i in cols:
        df[i] = (dta_df[i]- mean_dim[idx])*1.0/std_dim[idx]
        idx=idx+1
        
    return df

In [4]:
# get training and testing data prepared

cxtrain, dxtrain = conti_cate_split(xtrain_df, 106)
cxtest, dxtest = conti_cate_split(xtest_df, 106)

cxtest = conti_normalization_test_dta(cxtest, cxtrain)
cxtrain = conti_normalization_train_dta(cxtrain)

print cxtrain.shape, dxtrain.shape
print cxtest.shape, dxtest.shape

# cxtrain = cxtrain.as_matrix() 
dxtrain = dxtrain.as_matrix().astype(int)
cxtest = cxtest.as_matrix()
dxtest = dxtest.as_matrix().astype(int)

ytest = np_utils.to_categorical( ytest_df.as_matrix() )
ytrain =np_utils.to_categorical( ytrain_df.as_matrix() )

(6938, 106) (6938, 80)
(771, 106) (771, 80)


In [9]:
# only embedding

class mlp_demo():
    
    def __init__(self, batch_size, n_conti, n_disc, n_class,\
                 n_disc_voca, n_embedding, session, n_hidden_list, lr, l2):
        
        
        self.LEARNING_RATE = lr
                
        self.N_CLASS = n_class
        self.N_CONTI = n_conti
        self.N_DISC = n_disc
        self.N_DISC_VOCA = n_disc_voca
        self.N_EMBED = n_embedding
        self.L2 = l2
        
        self.N_HIDDEN_LAYERS = len(n_hidden_list) 
        self.N_TOTAL = self.N_CONTI+ self.N_DISC*self.N_EMBED
        
        self.sess=session
        
        self.dx = tf.placeholder(tf.int32, [None, self.N_DISC])
        self.cx = tf.placeholder(tf.float32, [None, self.N_CONTI])
        self.y = tf.placeholder(tf.float32, [None, self.N_CLASS])
        self.lr = tf.placeholder(tf.float32, shape=())
        self.keep_prob = tf.placeholder(tf.float32)
        
#       embedding categorical features
        self.dx_trans = tf.transpose( self.dx, [1,0] )
        
        tmp_embeded = []                
        for i in range(self.N_DISC):
            
            with tf.variable_scope("disc"+str(i)):
                w= tf.Variable(\
                        tf.random_normal([self.N_DISC_VOCA, self.N_EMBED],\
                        stddev=math.sqrt(2.0/float(self.N_DISC_VOCA))) )
#                     tf.random_uniform([self.N_DISC_VOCA, self.N_EMBED],-1.0, 1.0 )
                              
                tmp_embeded.append( tf.nn.embedding_lookup( w, self.dx_trans[i] ) )
                
        dx_embeded = tf.concat(tmp_embeded, 1)
        x_concate =  tf.concat( [self.cx,dx_embeded], 1 )

#----unit test---------------------------------------------------------
#         print self.N_TOTAL
#         self.tmpshape= tf.shape(x_concate)
        
#         self.tmpshape1= tf.shape(tmp_embeded[0])
# ---------------------------------------------------------------------


# #     hidden layers
        with tf.variable_scope("h0"):
            w= tf.Variable(tf.random_normal([self.N_TOTAL, n_hidden_list[0]],\
                        stddev=math.sqrt(2.0/float(self.N_TOTAL)))) 
            
            b= tf.Variable(tf.zeros( [ n_hidden_list[0] ] ))
            
            h = tf.nn.relu( tf.add( tf.matmul(x_concate, w),b) )
        
        
        for i in range(1, self.N_HIDDEN_LAYERS):
            
            with tf.variable_scope("layer"+str(i)):
                
                w= tf.Variable(tf.random_normal([n_hidden_list[i-1],n_hidden_list[i]],\
                                stddev=math.sqrt(2.0/float(n_hidden_list[i-1])))) 
                
                b= tf.Variable(tf.zeros( [n_hidden_list[i]] ))
                
                h = tf.nn.relu( tf.add( tf.matmul(h, w),b) )
        
#   Regularization  
#       dropout
        h = tf.nn.dropout(h, self.keep_prob)

#       L2
        self.regularizer = tf.nn.l2_loss(w)
    
        
#       output layer  
        with tf.variable_scope("output"):
            
            w= tf.Variable(tf.random_normal([n_hidden_list[self.N_HIDDEN_LAYERS-1],\
                                             self.N_CLASS],\
            stddev=math.sqrt(2.0/float(n_hidden_list[self.N_HIDDEN_LAYERS-1])))) 
                
            b= tf.Variable(tf.zeros( [ self.N_CLASS ] ))
            
            h = tf.add( tf.matmul(h, w),b)
            
#           L2  
            self.regularizer = self.regularizer + tf.nn.l2_loss(w)
            
            
            self.logit = h
    
#   initialize loss and optimization operations for training
    def train_ini(self):
        
        self.cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(\
                                                    logits=self.logit, labels=self.y) \
                                   )
#                                   + self.L2*self.regularizer)
        
        self.optimizer = \
        tf.train.AdadeltaOptimizer(learning_rate = self.LEARNING_RATE).minimize(self.cost)
#         tf.train.AdamOptimizer(learning_rate = self.lr).minimize(self.cost)
#         tf.train.GradientDescentOptimizer(learning_rate = self.lr).minimize(self.cost)
        
        self.init = tf.global_variables_initializer()
        sess.run( self.init )
        
#   training on batch of data
    def train_batch(self, dx_batch,cx_batch, y_batch, keep_prob, lr):
        
        _,c = sess.run([self.optimizer,self.cost],\
                       feed_dict={self.dx:dx_batch, \
                                  self.cx:cx_batch, self.y:y_batch,\
                                  self.lr:lr,\
                                  self.keep_prob:keep_prob\
                                 })
        
        return c
    
#   initialize inference         
    def inference_ini(self):
        
        self.correct_prediction = tf.equal(tf.argmax(self.logit,1), tf.argmax(self.y,1))
        self.accuracy = tf.reduce_mean(tf.cast(self.correct_prediction, "float"))
        
#   infer givn testing data    
    def inference(self, dx_test, cx_test, y_test, keep_prob):
        return sess.run([self.accuracy], feed_dict={self.dx:dx_test,\
                                                  self.cx:cx_test, self.y:y_test,\
                                                  self.keep_prob:keep_prob\
                                                   })
        
#   unit_test
    def test(self, dx_test, cx_test, y_test ):
        
        self.init = tf.global_variables_initializer()
        sess.run( self.init )
        
        return sess.run( [self.tmpshape], \
                 feed_dict={self.dx:dx_test, self.cx:cx_test, self.y:y_test})


In [12]:
para_n_epoch = 1000

# tunable parameters
# representation ability
para_n_hidden_list = [ 256,128,32 ]
para_n_embedding = 2
para_lr=0.007
# regularization
para_batch_size = 50
para_keep_prob=0.9
para_l2= 0.1


# fixed parameters
para_n_class = 8
para_n_disc = 80
para_n_conti = 106
para_n_disc_voca = 8

tot_cnt= np.shape(cxtrain)[0]


with tf.Session() as sess:
    
    mlp_clf = mlp_demo( para_batch_size, para_n_conti, para_n_disc, para_n_class, \
                        para_n_disc_voca, para_n_embedding, sess, para_n_hidden_list,\
                        para_lr, para_l2 )
    mlp_clf.train_ini()
    mlp_clf.inference_ini()
    
    para_cur_lr=para_lr
    total_batch = int(tot_cnt/para_batch_size)

    tot_idx=range(tot_cnt)

    
    # Training cycle
    for epoch in range(para_n_epoch):
        
        tmpc=0.0
        
#       shuffle traning dataset each epoch  
        np.random.shuffle(tot_idx)
   
        # Loop over all batches
        for i in range(total_batch):
            
            batch_idx = tot_idx[ i*para_batch_size: (i+1)*para_batch_size ] 
            
            batch_cx = cxtrain[ batch_idx ]
            batch_dx = dxtrain[ batch_idx ]
            batch_y  =  ytrain[ batch_idx ]
            
            tmpc += mlp_clf.train_batch( batch_dx, batch_cx, batch_y, para_keep_prob,\
                                       para_cur_lr)
        
#       learning rate decaying
        para_cur_lr = para_cur_lr*0.99
        
        tmp_test_acc = mlp_clf.inference(dxtest, cxtest, ytest, para_keep_prob) 
        tmp_train_acc = mlp_clf.inference(dxtrain, cxtrain, ytrain, para_keep_prob) 
        
        print "loss on epoch ", epoch, " : ", 1.0*tmpc/total_batch, tmp_test_acc,\
        tmp_train_acc
    
    print "Optimization Finished!"

loss on epoch  0  :  2.61422722582 [0.08690013] [0.084462382]
loss on epoch  1  :  2.49386830606 [0.10246433] [0.087633327]
loss on epoch  2  :  2.40369938249 [0.10505836] [0.097002015]
loss on epoch  3  :  2.32592620193 [0.12581064] [0.11790141]
loss on epoch  4  :  2.26486413375 [0.13618676] [0.13058518]
loss on epoch  5  :  2.21472775159 [0.15953307] [0.14153935]
loss on epoch  6  :  2.17865108234 [0.16990921] [0.16215047]
loss on epoch  7  :  2.14756187321 [0.17509727] [0.17541078]
loss on epoch  8  :  2.10958231878 [0.18677042] [0.18261747]
loss on epoch  9  :  2.08552519567 [0.18028535] [0.18535601]
loss on epoch  10  :  2.06685491403 [0.19844358] [0.19991352]
loss on epoch  11  :  2.04117549243 [0.19584955] [0.20265207]
loss on epoch  12  :  2.02940414263 [0.20492867] [0.21274142]
loss on epoch  13  :  2.01158284709 [0.1919585] [0.21202075]
loss on epoch  14  :  1.99941817574 [0.21919584] [0.22138944]
loss on epoch  15  :  1.98532693282 [0.21400778] [0.22268665]
loss on epoch  1

KeyboardInterrupt: 

In [84]:
# only wide
class wide_mlp_demo():
    
    def __init__(self, n_conti, n_disc, n_class,\
                 n_disc_voca, session, n_hidden_list, lr, l2):
        
        self.LEARNING_RATE = lr
                
        self.N_CLASS = n_class
        self.N_CONTI = n_conti
        self.N_DISC = n_disc
        self.N_DISC_VOCA = n_disc_voca
        self.L2 = l2
        
        self.N_HIDDEN_LAYERS = len(n_hidden_list) 
        self.sess=session
        
        self.dx = tf.placeholder(tf.int32, [None, self.N_DISC])
        self.cx = tf.placeholder(tf.float32, [None, self.N_CONTI])
        self.y = tf.placeholder(tf.float32, [None, self.N_CLASS])
        self.lr = tf.placeholder(tf.float32, shape=())
        self.keep_prob = tf.placeholder(tf.float32)
        
        
#       linnear part on categorical features
        self.dx_trans = tf.transpose( self.dx, [1,0] )
        
        for i in range(self.N_DISC):
            
            with tf.variable_scope("disc"+str(i)):
                w= tf.Variable(\
                        tf.random_uniform([self.N_DISC_VOCA, self.N_CLASS],-1.0, 1.0) )
#                         tf.random_normal([self.N_DISC_VOCA, self.N_CLASS],\
#                         stddev=math.sqrt(2.0/float(self.N_DISC_VOCA))) )
                
                if i==0:
                    dx_wsum = tf.nn.embedding_lookup( w, self.dx_trans[i] )
                else:
                    dx_wsum = dx_wsum + tf.nn.embedding_lookup( w, self.dx_trans[i] )
        
        with tf.variable_scope("wide"):
            b= tf.Variable(tf.zeros( [ self.N_CLASS ] ))            
            dx_wsum = tf.add(dx_wsum, b)

#----unit test---------------------------------------------------------
#         print self.N_TOTAL
#         self.tmpshape= tf.shape(x_concate)
        
#         self.tmpshape1= tf.shape(tmp_embeded[0])
# ---------------------------------------------------------------------

# #     hidden layers
        with tf.variable_scope("h0"):
            w= tf.Variable(tf.random_normal([self.N_CONTI, n_hidden_list[0]],\
                        stddev=math.sqrt(2.0/float(self.N_CONTI)))) 
            b= tf.Variable(tf.zeros( [ n_hidden_list[0] ] ))
            h = tf.nn.relu( tf.add( tf.matmul(self.cx, w),b) )
        
                
        for i in range(1, self.N_HIDDEN_LAYERS):
            
            with tf.variable_scope("layer"+str(i)):
                w= tf.Variable(tf.random_normal([n_hidden_list[i-1],n_hidden_list[i]],\
                                stddev=math.sqrt(2.0/float(n_hidden_list[i-1])))) 
                b= tf.Variable(tf.zeros( [n_hidden_list[i]] ))
                h = tf.nn.relu( tf.add( tf.matmul(h, w),b) )
        
#       Regularization
#       dropout
#         h = tf.nn.dropout(h, self.keep_prob)
#       L2
        self.regularizer = tf.nn.l2_loss(w)
        
#       output layer  
        with tf.variable_scope("output"):
            
            w= tf.Variable(tf.random_normal([n_hidden_list[self.N_HIDDEN_LAYERS-1],\
                                             self.N_CLASS],\
            stddev=math.sqrt(2.0/float(n_hidden_list[self.N_HIDDEN_LAYERS-1])))) 
            b= tf.Variable(tf.zeros( [ self.N_CLASS ] ))
            h = tf.add( tf.matmul(h, w),b)
            
#           L2  
            self.regularizer = self.regularizer + tf.nn.l2_loss(w)
            
            self.logit = h + dx_wsum
    
#   initialize loss and optimization operations for training
    def train_ini(self):
        self.cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(\
                                                    logits=self.logit, labels=self.y) \
                                  + self.L2*self.regularizer)
        self.optimizer = \
        tf.train.AdamOptimizer(learning_rate = self.lr).minimize(self.cost)
#       tf.train.RMSPropOptimizer(learning_rate = self.LEARNING_RATE).minimize(self.cost)
#         tf.train.AdamOptimizer(learning_rate = self.lr).minimize(self.cost)
#         tf.train.RMSPropOptimizer(learning_rate = self.lr).minimize(self.cost)
#         tf.train.AdadeltaOptimizer(learning_rate = self.lr).minimize(self.cost)
#         tf.train.AdamOptimizer(learning_rate = self.lr).minimize(self.cost)
#         tf.train.GradientDescentOptimizer(learning_rate = self.lr).minimize(self.cost)
        
        self.init = tf.global_variables_initializer()
        sess.run( self.init )
        
#   training on batch of data
    def train_batch(self, dx_batch,cx_batch, y_batch, keep_prob, lr):
        
        _,c = sess.run([self.optimizer,self.cost],\
                       feed_dict={self.dx:dx_batch, \
                                  self.cx:cx_batch, self.y:y_batch,\
                                  self.lr:lr,\
                                  self.keep_prob:keep_prob\
                                 })
        
        return c
    
#   initialize inference         
    def inference_ini(self):
        self.correct_prediction = tf.equal(tf.argmax(self.logit,1), tf.argmax(self.y,1))
        self.accuracy = tf.reduce_mean(tf.cast(self.correct_prediction, "float"))
        
#   infer givn testing data    
    def inference(self, dx_test, cx_test, y_test, keep_prob):
        return sess.run([self.accuracy], feed_dict={self.dx:dx_test,\
                                                  self.cx:cx_test, self.y:y_test,\
                                                  self.keep_prob:keep_prob\
                                                   })
        
#   unit_test
    def test(self, dx_test, cx_test, y_test ):
        self.init = tf.global_variables_initializer()
        sess.run( self.init )
        return sess.run( [self.tmpshape], \
                 feed_dict={self.dx:dx_test, self.cx:cx_test, self.y:y_test})


In [ ]:
para_n_epoch = 800

# tunable parameters
# representation ability
para_n_hidden_list = [ 16,8 ]
para_lr = 0.001
# regularization
para_batch_size = 128
para_keep_prob = 0.9
para_l2 = 0.1

# fixed parameters
para_n_class = 8
para_n_disc = 80
para_n_conti = 106
para_n_disc_voca = 8


tot_cnt= np.shape(cxtrain)[0]

with tf.Session() as sess:
    
    wide_mlp_clf = wide_mlp_demo( para_n_conti, para_n_disc, para_n_class, \
                        para_n_disc_voca, sess, para_n_hidden_list,\
                        para_lr, para_l2 )
    wide_mlp_clf.train_ini()
    wide_mlp_clf.inference_ini()
    
    para_cur_lr=para_lr
    total_batch = int(tot_cnt/para_batch_size)

    tot_idx=range(tot_cnt)

    
    # Training cycle
    for epoch in range(para_n_epoch):
        
        tmpc=0.0
        
#       shuffle traning dataset each epoch  
        np.random.shuffle(tot_idx)
   
        # Loop over all batches
        for i in range(total_batch):
            
            batch_idx = tot_idx[ i*para_batch_size: (i+1)*para_batch_size ] 
            
            batch_cx = cxtrain[ batch_idx ]
            batch_dx = dxtrain[ batch_idx ]
            batch_y  =  ytrain[ batch_idx ]
            
            tmpc += wide_mlp_clf.train_batch( batch_dx, batch_cx, batch_y, para_keep_prob,\
                                       para_cur_lr)
        
#       learning rate decaying
        para_cur_lr = para_cur_lr*0.98
        
        tmp_test_acc = wide_mlp_clf.inference(dxtest, cxtest, ytest, para_keep_prob) 
        tmp_train_acc = wide_mlp_clf.inference(dxtrain, cxtrain, ytrain, para_keep_prob) 
        
        print "loss on epoch ", epoch, " : ", 1.0*tmpc/total_batch, tmp_test_acc,\
        tmp_train_acc
    
    print "Optimization Finished!"

In [5]:
# wide and embedding
class wide_embed_NN():
    
    def __init__(self, n_conti, n_disc, n_class,\
                 n_disc_voca, session,n_embedding, n_hidden_list, lr, l2):
        
        self.LEARNING_RATE = lr
                
        self.N_CLASS = n_class
        self.N_CONTI = n_conti
        self.N_DISC = n_disc
        self.N_DISC_VOCA = n_disc_voca
        self.N_EMBED = n_embedding
        self.L2 = l2
        
        self.N_HIDDEN_LAYERS = len(n_hidden_list)
        self.N_TOTAL = self.N_CONTI+ self.N_DISC*self.N_EMBED

        self.sess=session
        
        self.dx = tf.placeholder(tf.int32, [None, self.N_DISC])
        self.cx = tf.placeholder(tf.float32, [None, self.N_CONTI])
        self.y = tf.placeholder(tf.float32, [None, self.N_CLASS])
        self.lr = tf.placeholder(tf.float32, shape=())
        self.keep_prob = tf.placeholder(tf.float32)
        
#       wide part on categorical features
        self.dx_trans = tf.transpose( self.dx, [1,0] )
        
        for i in range(self.N_DISC):
            
            with tf.variable_scope("wide"+str(i)):
                w= tf.Variable(\
                        tf.random_uniform([self.N_DISC_VOCA, self.N_CLASS],-1.0, 1.0) )
#                         tf.random_normal([self.N_DISC_VOCA, self.N_CLASS],\
#                         stddev=math.sqrt(2.0/float(self.N_DISC_VOCA))) )
                if i==0:
                    dx_wsum = tf.nn.embedding_lookup( w, self.dx_trans[i] )
                    
#                   L2
                    self.regularizer_wide = tf.nn.l2_loss(w)
                    
                else:
                    dx_wsum = dx_wsum + tf.nn.embedding_lookup( w, self.dx_trans[i] )
                    
#                   L2  
                    self.regularizer_wide = self.regularizer_wide + tf.nn.l2_loss(w)
                    
#       nonlinear    
        dx_wsum = tf.nn.relu( dx_wsum )

            
#       embedding categorical features
#         self.dx_trans = tf.transpose( self.dx, [1,0] )
        
        tmp_embeded = []                
        for i in range(self.N_DISC):
            
            with tf.variable_scope("embedding"+str(i)):
                w= tf.Variable(\
                        tf.random_normal([self.N_DISC_VOCA, self.N_EMBED],\
                        stddev=math.sqrt(2.0/float(self.N_DISC_VOCA))) )
#                     tf.random_uniform([self.N_DISC_VOCA, self.N_EMBED],-1.0, 1.0 )
                              
                tmp_embeded.append( tf.nn.embedding_lookup( w, self.dx_trans[i] ) )
                
        dx_embeded = tf.concat(tmp_embeded, 1)
        x_concate =  tf.concat( [self.cx,dx_embeded], 1 )

#----unit test---------------------------------------------------------
#         print self.N_TOTAL
#         self.tmpshape= tf.shape(x_concate)
        
#         self.tmpshape1= tf.shape(tmp_embeded[0])
# ---------------------------------------------------------------------

# #     hidden layers
        with tf.variable_scope("h0"):
            w= tf.Variable(tf.random_normal([self.N_TOTAL, n_hidden_list[0]],\
                        stddev=math.sqrt(2.0/float(self.N_CONTI)))) 
            b= tf.Variable(tf.zeros( [ n_hidden_list[0] ] ))
            h = tf.nn.relu( tf.add( tf.matmul(x_concate, w),b) )
        
                
        for i in range(1, self.N_HIDDEN_LAYERS):
            
            with tf.variable_scope("layer"+str(i)):
                w= tf.Variable(tf.random_normal([n_hidden_list[i-1],n_hidden_list[i]],\
                                stddev=math.sqrt(2.0/float(n_hidden_list[i-1])))) 
                b= tf.Variable(tf.zeros( [n_hidden_list[i]] ))
                h = tf.nn.relu( tf.add( tf.matmul(h, w),b) )
                
                
#       Regularization

#       dropout
#         if not (abs(self.keep_prob - 1.0) <= 1e-4):
#             h = tf.nn.dropout(h, self.keep_prob)
#       L2
        self.regularizer = tf.nn.l2_loss(w)
        
        
#       output layer  
        with tf.variable_scope("output"):
            
            w= tf.Variable(tf.random_normal([n_hidden_list[self.N_HIDDEN_LAYERS-1],\
                                             self.N_CLASS],\
            stddev=math.sqrt(2.0/float(n_hidden_list[self.N_HIDDEN_LAYERS-1])))) 
            b= tf.Variable(tf.zeros( [ self.N_CLASS ] ))
            
            
#             h = tf.add( tf.matmul(h, w), b )
            
            h = tf.add( tf.matmul(h, w), dx_wsum )
            h = tf.add( h, b )
#             
            self.logit = h
     
    
#           L2  
            self.regularizer = self.regularizer + tf.nn.l2_loss(w)
    
#       overall L2
        self.regularizer= self.regularizer + self.regularizer_wide
         
    
#   initialize loss and optimization operations for training
    def train_ini(self):
        self.cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(\
                                                    logits=self.logit, labels=self.y) \
                                  + self.L2*self.regularizer )
        self.optimizer = \
        tf.train.AdadeltaOptimizer(learning_rate = self.LEARNING_RATE).minimize(self.cost)
#       tf.train.RMSPropOptimizer(learning_rate = self.LEARNING_RATE).minimize(self.cost)
#         tf.train.AdamOptimizer(learning_rate = self.lr).minimize(self.cost)
#         tf.train.RMSPropOptimizer(learning_rate = self.lr).minimize(self.cost)
#         tf.train.AdadeltaOptimizer(learning_rate = self.lr).minimize(self.cost)
#         tf.train.AdamOptimizer(learning_rate = self.lr).minimize(self.cost)
#         tf.train.GradientDescentOptimizer(learning_rate = self.lr).minimize(self.cost)
        
        self.init = tf.global_variables_initializer()
        sess.run( self.init )
        
#   training on batch of data
    def train_batch(self, dx_batch,cx_batch, y_batch, keep_prob, lr):
        
        _,c = sess.run([self.optimizer,self.cost],\
                       feed_dict={self.dx:dx_batch, \
                                  self.cx:cx_batch, self.y:y_batch,\
                                  self.lr:lr,\
                                  self.keep_prob:keep_prob\
                                 })
        
        return c
    
#   initialize inference         
    def inference_ini(self):
        self.correct_prediction = tf.equal(tf.argmax(self.logit,1), tf.argmax(self.y,1))
        self.accuracy = tf.reduce_mean(tf.cast(self.correct_prediction, "float"))
        
#   infer givn testing data    
    def inference(self, dx_test, cx_test, y_test, keep_prob):
        return sess.run([self.accuracy], feed_dict={self.dx:dx_test,\
                                                  self.cx:cx_test, self.y:y_test,\
                                                  self.keep_prob:keep_prob\
                                                   })
        
#   unit_test
    def test(self, dx_test, cx_test, y_test ):
        self.init = tf.global_variables_initializer()
        sess.run( self.init )
        return sess.run( [self.tmpshape], \
                 feed_dict={self.dx:dx_test, self.cx:cx_test, self.y:y_test})


In [ ]:
para_n_epoch = 1200

# tunable parameters
#   representation ability
para_n_hidden_list = [ 128,64,8 ]
#     128,16,8 ]
para_lr = 0.045
# 0.05
para_n_embedding = 2
#   regularization
para_batch_size = 32
para_keep_prob = 1.0
para_l2 = 0.3
# 0.15

# fixed parameters
para_n_class = 8
para_n_disc = 80
para_n_conti = 106
para_n_disc_voca = 8

tot_cnt= np.shape(cxtrain)[0]


with tf.Session() as sess:
    
    wide_mlp_clf = wide_embed_NN( para_n_conti, para_n_disc, para_n_class, \
                      para_n_disc_voca, sess, para_n_embedding, para_n_hidden_list,\
                        para_lr, para_l2 )
    wide_mlp_clf.train_ini()
    wide_mlp_clf.inference_ini()
    
    para_cur_lr=para_lr
    total_batch = int(tot_cnt/para_batch_size)

    tot_idx=range(tot_cnt)

    
    # Training cycle
    for epoch in range(para_n_epoch):
        
        tmpc=0.0
        
#       shuffle traning dataset each epoch  
        np.random.shuffle(tot_idx)
   
        # Loop over all batches
        for i in range(total_batch):
            
            batch_idx = tot_idx[ i*para_batch_size: (i+1)*para_batch_size ] 
            
            batch_cx = cxtrain[ batch_idx ]
            batch_dx = dxtrain[ batch_idx ]
            batch_y  =  ytrain[ batch_idx ]
            
            tmpc += wide_mlp_clf.train_batch( batch_dx, batch_cx, batch_y, para_keep_prob,\
                                       para_cur_lr)
        
#       learning rate decaying
#         para_cur_lr = para_cur_lr*0.98
        
        tmp_test_acc = wide_mlp_clf.inference(dxtest, cxtest, ytest, para_keep_prob) 
        tmp_train_acc = wide_mlp_clf.inference(dxtrain, cxtrain, ytrain, para_keep_prob) 
        
        print "loss on epoch ", epoch, " : ", 1.0*tmpc/total_batch, tmp_test_acc,\
        tmp_train_acc
    
    print "Optimization Finished!"

loss on epoch  0  :  266.250372286 [0.1465629] [0.14557509]
loss on epoch  1  :  261.815340819 [0.14915694] [0.14701644]
loss on epoch  2  :  256.887796402 [0.15564202] [0.14889017]
loss on epoch  3  :  251.546624996 [0.15823606] [0.15191698]
loss on epoch  4  :  245.874232751 [0.15953307] [0.15249351]
loss on epoch  5  :  239.944819627 [0.16601816] [0.15624098]
loss on epoch  6  :  233.809127525 [0.16342412] [0.16070914]
loss on epoch  7  :  227.512261921 [0.16990921] [0.16474488]
loss on epoch  8  :  221.098424276 [0.1763943] [0.16849236]
loss on epoch  9  :  214.603237788 [0.1841764] [0.17368117]
loss on epoch  10  :  208.04042802 [0.18936446] [0.17858173]
loss on epoch  11  :  201.449299565 [0.19584955] [0.1801672]
loss on epoch  12  :  194.84480434 [0.19844358] [0.18319401]
loss on epoch  13  :  188.249538987 [0.19844358] [0.18665321]
loss on epoch  14  :  181.681809814 [0.19714656] [0.19025655]
loss on epoch  15  :  175.16517399 [0.20233463] [0.19184203]
loss on epoch  16  :  168